In [ ]:
%%sh 

algorithm_name='sklearn-boston-housing-mme'

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

ecr_image="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids ${account})

# Build the docker image locally with the image name and then push it to ECR
# with the full image name.

# First clear out any prior version of the cloned repo
rm -rf sagemaker-scikit-learn-container/

# Clone the sklearn container repo
git clone --single-branch --branch mme https://github.com/aws/sagemaker-scikit-learn-container.git
cd sagemaker-scikit-learn-container/

# Build the "base" container image that encompasses the installation of the
# scikit-learn framework and all of the dependencies needed.
docker build -q -t sklearn-base:0.20-2-cpu-py3 -f docker/0.20-2/base/Dockerfile.cpu --build-arg py_version=3 .

# Create the SageMaker Scikit-learn Container Python package.
python setup.py bdist_wheel --universal

# Build the "final" container image that encompasses the installation of the
# code that implements the SageMaker multi-model container requirements.
docker build -q -t ${algorithm_name} -f docker/0.20-2/final/Dockerfile.cpu .

docker tag ${algorithm_name} ${ecr_image}

docker push ${ecr_image}

In [ ]:
import sagemaker

In [ ]:
sess = sagemaker.Session()
bucket = sess.default_bucket()   
role   = sagemaker.get_execution_role()

prefix = 'sklearn-boston-housing-mme'

training = sess.upload_data(path='housing.csv', key_prefix=prefix + "/training")
output   = 's3://{}/{}/output/'.format(bucket,prefix)
print(training)
print(output)

In [ ]:
from sagemaker.sklearn import SKLearn

jobs =  {}
for test_size in [0.2, 0.1, 0.05]:
    sk = SKLearn(entry_point='sklearn-boston-housing.py',
                 role=role,
                 train_instance_count=1, 
                 train_instance_type='ml.m5.large',
                 output_path=output,
                 hyperparameters={
                      'normalize': True,
                      'test-size': test_size,
                  }
    )
    sk.fit({'training':training}, wait=False)
    jobs[sk.latest_training_job.name] = {}
    jobs[sk.latest_training_job.name]['test-size'] = test_size

In [ ]:
jobs

In [ ]:
import boto3
sm = boto3.client('sagemaker')

for j in jobs.keys():
    job = sm.describe_training_job(TrainingJobName=j)
    jobs[j]['artifact'] = job['ModelArtifacts']['S3ModelArtifacts']
    jobs[j]['key'] = '/'.join(job['ModelArtifacts']['S3ModelArtifacts'].split('/')[3:8])

In [ ]:
jobs

In [ ]:
%%sh -s "$bucket" "$prefix"
aws s3 rm --recursive s3://$1/$2/models

In [ ]:
s3 = boto3.client('s3')

for j in jobs.keys():
    copy_source = { 'Bucket': bucket, 'Key': jobs[j]['key'] }
    s3.copy_object(CopySource=copy_source, Bucket=bucket, Key=prefix+'/models/'+j+'.tar.gz')

response = s3.list_objects(Bucket=bucket, Prefix=prefix+'/models/')
for o in response['Contents']:
    print(o['Key'])

In [ ]:
script = 'sklearn-boston-housing.py'
script_archive = 's3://{}/{}/source/source.tar.gz'.format(bucket, prefix)

print(script)
print(script_archive)

In [ ]:
%%sh -s "$script" "$script_archive"
tar cvfz source.tar.gz $1
aws s3 cp source.tar.gz $2

In [ ]:
account = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, prefix)

print(container)

In [ ]:
response = sm.create_model(
    ModelName = prefix,
    ExecutionRoleArn = role,
    Containers = [
        {
            'Image': container,
            'ModelDataUrl': 's3://{}/{}/models/'.format(bucket, prefix),
            'Mode': 'MultiModel',
            'Environment': {
                'SAGEMAKER_PROGRAM' : script,
                'SAGEMAKER_SUBMIT_DIRECTORY' : script_archive
            }
        }
    ]
)

print(response)

In [ ]:
epc_name = prefix+'-epc'

response = sm.create_endpoint_config(
    EndpointConfigName = epc_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': prefix,
        'VariantName': 'variant-1'}
    ]
)

print(response)

In [ ]:
ep_name = prefix+'-ep'

response = sm.create_endpoint(
    EndpointName=ep_name,
    EndpointConfigName=epc_name)

print(response)

In [ ]:
sm.describe_endpoint(EndpointName=ep_name)

In [ ]:
waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=ep_name)

In [ ]:
import pandas as pd
import numpy as np
from io import BytesIO

data = pd.read_csv('housing.csv', delim_whitespace=True)
payload = data[:1].drop(['medv'], axis=1)
buffer = BytesIO()
np.save(buffer, payload.values)

In [ ]:
smrt = boto3.client('runtime.sagemaker')

for j in jobs.keys():
    model_name=j+'.tar.gz'
    print(model_name)
    response = smrt.invoke_endpoint(
        EndpointName=ep_name,
        TargetModel=model_name,
        Body=buffer.getvalue(),
        ContentType='application/x-npy')

    print(response['Body'].read())

In [ ]:
sk = SKLearn(entry_point='sklearn-boston-housing.py',
                 role=role,
                 train_instance_count=1, 
                 train_instance_type='ml.m5.large',
                 output_path=output,
                 hyperparameters={
                      'normalize': False,
                      'test-size': 0.15,
                  }
)

sk.fit({'training':training})

In [ ]:
job = sm.describe_training_job(TrainingJobName=sk.latest_training_job.name)
job_name = sk.latest_training_job.name
artifact = job['ModelArtifacts']['S3ModelArtifacts']

In [ ]:
%%sh -s "$artifact" "$bucket" "$prefix" "$job_name"
aws s3 cp $1 s3://$2/$3/models/$4.tar.gz
aws s3 ls s3://$2/$3/models/

In [ ]:
model_name=job_name+'.tar.gz'
print(model_name)

response = smrt.invoke_endpoint(
    EndpointName=ep_name,
    TargetModel=model_name,
    Body=buffer.getvalue(),
    ContentType='application/x-npy')

print(response['Body'].read())

In [ ]:
sm.delete_endpoint(EndpointName=ep_name)

In [ ]:
sm.delete_endpoint_config(EndpointConfigName=epc_name)

In [ ]:
sm.delete_model(ModelName=prefix)